<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Sparse_BIC_18_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (sparse)

In [ ]:
# Function to generate CPDs for the sparse structure with 18 nodes
def generate_cpds_sparse():
    # Generate random probabilities for each node independently
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()

    irt_given_ir_probs = np.random.rand(3, 3)
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    ei_given_ir_irt_probs = np.random.rand(3, 3, 3)
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    ue_given_geo_probs = np.random.rand(3, 3)
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    ms_given_ei_geo_probs = np.random.rand(3, 3, 3)
    ms_given_ei_geo_probs /= ms_given_ei_geo_probs.sum(axis=0, keepdims=True)

    irp_given_ir_irt_probs = np.random.rand(3, 3, 3)
    irp_given_ir_irt_probs /= irp_given_ir_irt_probs.sum(axis=0, keepdims=True)

    inf_given_ms_irp_probs = np.random.rand(3, 3, 3)
    inf_given_ms_irp_probs /= inf_given_ms_irp_probs.sum(axis=0, keepdims=True)

    # Additional independent nodes
    gdp_probs = np.random.rand(3)
    gdp_probs /= gdp_probs.sum()

    inv_probs = np.random.rand(3)
    inv_probs /= inv_probs.sum()

    ci_probs = np.random.rand(3)
    ci_probs /= ci_probs.sum()

    oil_probs = np.random.rand(3)
    oil_probs /= oil_probs.sum()

    fx_probs = np.random.rand(3)
    fx_probs /= fx_probs.sum()

    irb_probs = np.random.rand(3)
    irb_probs /= irb_probs.sum()

    # New nodes: POP, TECH, and ENRG
    pop_probs = np.random.rand(3)
    pop_probs /= pop_probs.sum()

    tech_probs = np.random.rand(3)
    tech_probs /= tech_probs.sum()

    enrg_probs = np.random.rand(3)
    enrg_probs /= enrg_probs.sum()

    # SP depends sparsely on each node without interactions between them
    sp_probs = np.random.rand(3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return (ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs,
            ms_given_ei_geo_probs, irp_given_ir_irt_probs, inf_given_ms_irp_probs, gdp_probs,
            inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, pop_probs, tech_probs, enrg_probs, sp_probs)

# Function to generate and save samples with the updated sparse structure
def generate_and_save_samples_sparse(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs,
                                     irp_probs, inf_probs, gdp_probs, inv_probs, ci_probs, oil_probs,
                                     fx_probs, irb_probs, pop_probs, tech_probs, enrg_probs, sp_probs,
                                     sample_size, filename):
    output_data = []

    for _ in range(sample_size):
        # Sample each node individually
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]

        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]

        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]

        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]

        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]

        ms_probs_given_ei_geo = ms_probs[:, ei_state_idx, geo_state_idx]
        ms_state_idx = np.random.choice(3, p=ms_probs_given_ei_geo)
        ms_state = ['negative', 'neutral', 'positive'][ms_state_idx]

        irp_probs_given_ir_irt = irp_probs[:, ir_state_idx, irt_state_idx]
        irp_state_idx = np.random.choice(3, p=irp_probs_given_ir_irt)
        irp_state = ['loose', 'neutral', 'tight'][irp_state_idx]

        inf_probs_given_ms_irp = inf_probs[:, ms_state_idx, irp_state_idx]
        inf_state_idx = np.random.choice(3, p=inf_probs_given_ms_irp)
        inf_state = ['deflation', 'stable', 'inflation'][inf_state_idx]

        gdp_state_idx = np.random.choice(3, p=gdp_probs)
        gdp_state = ['low', 'medium', 'high'][gdp_state_idx]

        inv_state_idx = np.random.choice(3, p=inv_probs)
        inv_state = ['decreasing', 'steady', 'increasing'][inv_state_idx]

        ci_state_idx = np.random.choice(3, p=ci_probs)
        ci_state = ['low', 'medium', 'high'][ci_state_idx]

        oil_state_idx = np.random.choice(3, p=oil_probs)
        oil_state = ['low', 'medium', 'high'][oil_state_idx]

        fx_state_idx = np.random.choice(3, p=fx_probs)
        fx_state = ['weak', 'stable', 'strong'][fx_state_idx]

        irb_state_idx = np.random.choice(3, p=irb_probs)
        irb_state = ['loose', 'neutral', 'tight'][irb_state_idx]

        pop_state_idx = np.random.choice(3, p=pop_probs)
        pop_state = ['low', 'medium', 'high'][pop_state_idx]

        tech_state_idx = np.random.choice(3, p=tech_probs)
        tech_state = ['slow', 'moderate', 'fast'][tech_state_idx]

        enrg_state_idx = np.random.choice(3, p=enrg_probs)
        enrg_state = ['scarce', 'adequate', 'abundant'][enrg_state_idx]

        # Calculate SP probability based on the state of each node (sparse dependency on each)
        sp_probs_given_all = sp_probs[:, ir_state_idx, irt_state_idx, ei_state_idx, geo_state_idx,
                                      ue_state_idx, ms_state_idx, irp_state_idx, inf_state_idx,
                                      gdp_state_idx, inv_state_idx, ci_state_idx, oil_state_idx,
                                      fx_state_idx, irb_state_idx, pop_state_idx, tech_state_idx,
                                      enrg_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_all)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]

        # Append sample data to output list including probabilities for all nodes
        output_data.append({
            'IR_State': ir_state,
            'IRT_State': irt_state,
            'EI_State': ei_state,
            'GEO_State': geo_state,
            'UE_State': ue_state,
            'MS_State': ms_state,
            'IRP_State': irp_state,
            'INF_State': inf_state,
            'GDP_State': gdp_state,
            'INV_State': inv_state,
            'CI_State': ci_state,
            'OIL_State': oil_state,
            'FX_State': fx_state,
            'IRB_State': irb_state,
            'POP_State': pop_state,
            'TECH_State': tech_state,
            'ENRG_State': enrg_state,
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_all]),
            'Chosen_SP_State': sp_state
        })

    output_df = pd.DataFrame(output_data)
    output_df.to_csv(filename, index=False)

    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 150, 200, 500, 1000, 5000, 10000, 15000, 20000]

for size in sample_sizes:
    ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, gdp_probs, \
    inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, pop_probs, tech_probs, enrg_probs, sp_probs = generate_cpds_sparse()

    generate_and_save_samples_sparse(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs,
                                     irp_probs, inf_probs, gdp_probs, inv_probs, ci_probs, oil_probs,
                                     fx_probs, irb_probs, pop_probs, tech_probs, enrg_probs, sp_probs,
                                     size, f'combined_probabilities_sparse_{size}.csv')

print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-------------+------------+-------------+------------+------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+-------------------------------------------------+-------------------+
|    | IR_State   | IRT_State   | EI_State   | GEO_State   | UE_State   | MS_State   | IRP_State   | INF_State   | GDP_State   | INV_State   | CI_State   | OIL_State   | FX_State   | IRB_State   | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |
+====+============+=============+============+=============+============+============+=============+=============+=============+=============+============+=============+============+=============+=================================================+===================+
|  0 | high       | decreasing  | poor       | stable      | medium     | negative   | loose       | stable      | high        | increasing  |

# LBN Dense BIC & Entropy

In [ ]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 500, 1000, 5000, 10000, 15000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'combined_probabilities_sparse_{sample_size}.csv'
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for each of the 18 nodes
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    ms_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    irp_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    inf_map = {'deflation': 0, 'stable': 1, 'inflation': 2}
    gdp_map = {'low': 0, 'medium': 1, 'high': 2}
    inv_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ci_map = {'low': 0, 'medium': 1, 'high': 2}
    oil_map = {'low': 0, 'medium': 1, 'high': 2}
    fx_map = {'weak': 0, 'stable': 1, 'strong': 2}
    irb_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    pop_map = {'low': 0, 'medium': 1, 'high': 2}
    tech_map = {'slow': 0, 'moderate': 1, 'fast': 2}
    enrg_map = {'scarce': 0, 'adequate': 1, 'abundant': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    # Encode each node in the DataFrame
    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['MS_encoded'] = df['MS_State'].map(ms_map)
    df['IRP_encoded'] = df['IRP_State'].map(irp_map)
    df['INF_encoded'] = df['INF_State'].map(inf_map)
    df['GDP_encoded'] = df['GDP_State'].map(gdp_map)
    df['INV_encoded'] = df['INV_State'].map(inv_map)
    df['CI_encoded'] = df['CI_State'].map(ci_map)
    df['OIL_encoded'] = df['OIL_State'].map(oil_map)
    df['FX_encoded'] = df['FX_State'].map(fx_map)
    df['IRB_encoded'] = df['IRB_State'].map(irb_map)
    df['POP_encoded'] = df['POP_State'].map(pop_map)
    df['TECH_encoded'] = df['TECH_State'].map(tech_map)
    df['ENRG_encoded'] = df['ENRG_State'].map(enrg_map)
    df['SP_encoded'] = df['Chosen_SP_State'].map(sp_map)

    # Split the data into training and test sets
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded', 'MS_encoded',
            'IRP_encoded', 'INF_encoded', 'GDP_encoded', 'INV_encoded', 'CI_encoded',
            'OIL_encoded', 'FX_encoded', 'IRB_encoded', 'POP_encoded', 'TECH_encoded', 'ENRG_encoded']]
    y = df['SP_encoded']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

    # Concatenate X_train and y_train to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)
    df_test = pd.concat([X_test, y_test], axis=1)

    # Define the sparse structure with priors (fixed edges only pointing to SP)
    priors = [
        ('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded'),
        ('GEO_encoded', 'SP_encoded'), ('UE_encoded', 'SP_encoded'), ('MS_encoded', 'SP_encoded'),
        ('IRP_encoded', 'SP_encoded'), ('INF_encoded', 'SP_encoded'), ('GDP_encoded', 'SP_encoded'),
        ('INV_encoded', 'SP_encoded'), ('CI_encoded', 'SP_encoded'), ('OIL_encoded', 'SP_encoded'),
        ('FX_encoded', 'SP_encoded'), ('IRB_encoded', 'SP_encoded'), ('POP_encoded', 'SP_encoded'),
        ('TECH_encoded', 'SP_encoded'), ('ENRG_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with priors on the training data
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator on the training data
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # Predict on test data and calculate K-L divergence
    try:
        # Generate predictions for SP_encoded based on the learned model
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_sparse_18nodes.csv', index=False)

print("\nAll results have been saved.")